In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [2]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathVideo =  configParser.get('COMMON', 'test_datasetPathVideo')
datasetPathFaces =  configParser.get('evaluate_imagen', 'test_datasetPathFaces')
datasetPathFeatures =  configParser.get('evaluate_imagen', 'test_datasetPathFeatures')
datasetPathGuides =  configParser.get('evaluate_imagen', 'test_datasetPathGuides')

datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'

ttwbdf =  int(configParser.get('evaluate_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [3]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [4]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [5]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [6]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
timesteps= int(configParser.get('COMMON', 'timesteps') )
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')
folder =  configParser.get('evaluate_imagen', 'folder')

In [7]:

boxBlurMin =  int(configParser.get('evaluate_imagen', 'boxBlurMin'))
boxBlurMax =  int(configParser.get('evaluate_imagen', 'boxBlurMax'))

gaussianBlurMin =  int(configParser.get('evaluate_imagen', 'gaussianBlurMin'))
gaussianBlurMax =  int(configParser.get('evaluate_imagen', 'gaussianBlurMax'))

pix_to_min =  int(configParser.get('evaluate_imagen', 'pix_to_min'))
pix_to_max =  int(configParser.get('evaluate_imagen', 'pix_to_max'))

fddfb =  int(configParser.get('evaluate_imagen', 'faceDetectionDeepFaceBackend'))
efvr =  float(configParser.get('evaluate_imagen', 'expandFaceVerticalRatio'))
efhr =  float(configParser.get('evaluate_imagen', 'expandFaceHorizontalRatio'))


In [8]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table = db.open_table("video")

In [9]:
videos = table.to_pandas()

In [10]:
videos_unique_users = videos
videos_unique_users


,id,video_path,face_path,user,blurred_face_path,features_path,vector,stage
0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id00017,None,None,[0.47388548],None
1,1,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id00061,None,None,[0.9108679],None
2,2,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id00081,None,None,[0.17406929],None
3,3,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id00154,None,None,[0.6630942],None
4,4,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id00419,None,None,[0.70828676],None
...,...,...,...,...,...,...,...,...
113,113,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id08552,None,None,[0.5701425],None
114,114,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id08696,None,None,[0.7607498],None
115,115,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id08701,None,None,[0.16722195],None
116,116,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,None,id08911,None,None,[0.9646658],None


In [11]:
try:
  db.drop_table("video_stage1")
except:
  pass

In [12]:
import multiprocessing
try:
    multiprocessing.set_start_method('spawn')
except:
    pass

In [13]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
from testing_imagen_audio_features import extract_audio_features
from testing_imagen_vision_transformer import extract_vision_transformer
from testing_imagen_face import extract_face
import pathlib
import shutil


2023-11-07 10:06:03.791614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 10:06:03.837322: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[NeMo W 2023-11-07 10:06:05 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-11-07 10:06:05 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at you

In [14]:


i = 0
df_data = pd.DataFrame(columns=['index','row','absPathVideo','absPathFace'])
for index, row in videos_unique_users.iterrows():
    video_path = absPathVideo = row['video_path']


    # EXTRACT FACE, AGE, GENDER, ETHNICITY, FACE EMBEDDING
    absPathFace = absPathVideo.replace(datasetPathVideo,datasetPathFaces)
    absPathFace = os.path.splitext(absPathFace)[0]
    absPathFace = absPathFace + "_face_" + str(1) + ".png"
    #print(absPathFace)
    #Create Directory
    pathlib.Path(os.path.dirname(absPathFace)).mkdir(parents=True, exist_ok=True) 
    df_data = df_data.append({'index': index,'row': row,'absPathVideo':absPathVideo,'absPathFace':absPathFace }, ignore_index=True)


proc = multiprocessing.Process(target=extract_face, args=(df_data,
                                                        image_size,
                                                        fddfb,
                                                        output_folder,
                                                        efvr,efhr,True,10))
proc.start()
proc.join()

Action: race: 100%|██████████| 3/3 [00:00<00:00, 16.64it/s]  


In [15]:
with open(output_folder + '/' + 'df_data1.pickle', 'rb') as handle:
    df_data = pickle.load(handle)
df_data

,index,row,absPathVideo,absPathFace,gender,ethnicity,age,VGG
0,0,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,white,26,"[0.02598828449845314, 0.002286622067913413, 0...."
1,1,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,white,31,"[0.014909587800502777, 0.010321180336177349, 0..."
2,2,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,latino hispanic,28,"[0.017716582864522934, 0.031231218948960304, 0..."
3,3,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,middle eastern,29,"[0.015032665804028511, 0.0020783008076250553, ..."
4,4,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Woman,white,38,"[0.014704544097185135, -0.004270102363079786, ..."
...,...,...,...,...,...,...,...,...
113,113,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,black,26,"[0.011644644662737846, 0.006384784821420908, 0..."
114,114,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,black,27,"[0.012097470462322235, 0.011594198644161224, 0..."
115,115,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,white,44,"[0.020973164588212967, -0.011326933279633522, ..."
116,116,id ...,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,Man,white,26,"[0.027417417615652084, -0.00149383838288486, 0..."


In [16]:


dataGotten = pd.DataFrame(columns=['ID','FACE_ID','VIDEO_PATH','caption_a',
                                   'caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','AUDIO_FEATURES','caption_l',
                                   'image_path','VGG'])
dataGotten['ID'] = df_data['index']
dataGotten['FACE_ID'] = df_data['index']
dataGotten['VIDEO_PATH'] = df_data['absPathVideo']
dataGotten['caption_a'] = df_data['age']
dataGotten['caption_e'] = df_data['ethnicity']
dataGotten['caption_g'] = df_data['gender']
dataGotten['SPEAKER_EMB'] = np.nan
dataGotten['AUDIO_EMB'] = np.nan
dataGotten['AUDIO_FEATURES'] = np.nan
dataGotten['caption_l'] = df_data['age']
dataGotten['image_path'] = df_data['absPathFace']
dataGotten['VGG'] = df_data['VGG']
dataGotten["image_guide"] = np.nan
dataGotten['low_res_image'] = df_data['absPathFace']
dataGotten

,ID,FACE_ID,VIDEO_PATH,caption_a,caption_e,caption_g,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,caption_l,image_path,VGG,image_guide,low_res_image
0,0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.02598828449845314, 0.002286622067913413, 0....",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
1,1,1,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,31,white,Man,NaN,NaN,NaN,31,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.014909587800502777, 0.010321180336177349, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
2,2,2,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,28,latino hispanic,Man,NaN,NaN,NaN,28,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.017716582864522934, 0.031231218948960304, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
3,3,3,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,29,middle eastern,Man,NaN,NaN,NaN,29,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.015032665804028511, 0.0020783008076250553, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
4,4,4,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,38,white,Woman,NaN,NaN,NaN,38,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.014704544097185135, -0.004270102363079786, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,113,113,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,black,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.011644644662737846, 0.006384784821420908, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
114,114,114,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,27,black,Man,NaN,NaN,NaN,27,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.012097470462322235, 0.011594198644161224, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
115,115,115,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,44,white,Man,NaN,NaN,NaN,44,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.020973164588212967, -0.011326933279633522, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...
116,116,116,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.027417417615652084, -0.00149383838288486, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...


In [17]:
import pickle
audio_embs =  configParser.get('COMMON', 'audio_embs') 
from pydub import AudioSegment
import math
from threading import Thread
import time
import subprocess

dataGotten['path_var_len_audio'] = np.NaN

ftd = []
# Function to delete audio temp files
def delFiles(filesToDelete):
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass

def extract_path_var_len_audio_per_row(row):
    print(row['ID'])
    # EXTRACT SPEAKER EMBEDDINGS, AUDIO FEATURES AND LANGUAGE
    video_path = row['VIDEO_PATH']

    # Get original duration of video
    audio = AudioSegment.from_file(video_path)
    
    audio_length_og = math.floor(audio.duration_seconds)


    video_filename = os.path.basename(video_path)

    i = str(row['ID'])

    absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
    absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
    absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


    
    # Extract audio monochannel and with 16khz and put it in absPathAudio

    command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
    subprocess.call(command, shell=True)



    # Will either truncate or loop the original video to reach audio_length (6,12 or 24)
    audio_length = 24
    path_var_len_audio =  absPathAudio_w + "audio_" + i +'_' +  str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio
    #print(path_var_len_audio)
    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)
    ftd.extend([absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp])

    return path_var_len_audio
dataGotten['path_var_len_audio']= dataGotten.apply(extract_path_var_len_audio_per_row,args=(),axis=1)

dataGotten


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


,ID,FACE_ID,VIDEO_PATH,caption_a,caption_e,caption_g,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,caption_l,image_path,VGG,image_guide,low_res_image,path_var_len_audio
0,0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.02598828449845314, 0.002286622067913413, 0....",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
1,1,1,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,31,white,Man,NaN,NaN,NaN,31,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.014909587800502777, 0.010321180336177349, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
2,2,2,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,28,latino hispanic,Man,NaN,NaN,NaN,28,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.017716582864522934, 0.031231218948960304, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
3,3,3,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,29,middle eastern,Man,NaN,NaN,NaN,29,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.015032665804028511, 0.0020783008076250553, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
4,4,4,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,38,white,Woman,NaN,NaN,NaN,38,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.014704544097185135, -0.004270102363079786, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,113,113,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,black,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.011644644662737846, 0.006384784821420908, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
114,114,114,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,27,black,Man,NaN,NaN,NaN,27,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.012097470462322235, 0.011594198644161224, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
115,115,115,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,44,white,Man,NaN,NaN,NaN,44,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.020973164588212967, -0.011326933279633522, ...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...
116,116,116,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,NaN,NaN,NaN,26,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.027417417615652084, -0.00149383838288486, 0...",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...


In [18]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(dataGotten,output_folder,))
proc.start()
proc.join()
with open(output_folder + '/' + 'df_data2.pickle', 'rb') as handle:
    dataGotten = pickle.load(handle)

0


/home/gamal/anaconda3/envs/ds2f_pre/lib/python3.8/site-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


In [19]:
dataGotten.head(1)

,ID,FACE_ID,VIDEO_PATH,caption_a,caption_e,caption_g,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,caption_l,image_path,VGG,image_guide,low_res_image,path_var_len_audio
0,0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,NaN,NaN,English,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.02598828449845314, 0.002286622067913413, 0....",NaN,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...


In [20]:
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(dataGotten,output_folder,))
proc.start()
proc.join()
with open(output_folder + '/' + 'dataGotten3.pickle', 'rb') as handle:
    dataGotten = pickle.load(handle)


[NeMo W 2023-11-07 10:07:37 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-11-07 10:07:37 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/c6335d8f1cd77b30084387468a6cf26fea90009b/pytorch_model.bin`
Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/c6335d8f1cd77b30084387468a6cf26fea90009b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


[NeMo W 2023-11-07 10:07:39 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-11-07 10:07:39 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-11-07 10:07:39 features:287] PADDING: 16
[NeMo I 2023-11-07 10:07:39 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.
[NeMo I 2023-11-07 10:07:39 cloud:58] Found existing object /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-11-07 10:07:39 cloud:64] Re-using file from: /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-11-07 10:07:39 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-11-07 10:07:39 features:287] PADDING: 16


[NeMo W 2023-11-07 10:07:39 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-11-07 10:07:39 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-11-07 10:07:39 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


In [21]:
dataGotten['AUDIO_EMB']

0      b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
1      b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
2      b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
3      b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
4      b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
                             ...                        
113    b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
114    b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
115    b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
116    b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
117    b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
Name: AUDIO_EMB, Length: 118, dtype: object

In [22]:
from testing_imagen_audio_features import extract_audio_features
proc = multiprocessing.Process(target=extract_audio_features, args=(dataGotten,output_folder,))
proc.start()
proc.join()
with open(output_folder + '/' + 'dataGotten4.pickle', 'rb') as handle:
    dataGotten = pickle.load(handle)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


In [23]:
dataGotten['AUDIO_FEATURES']

0      b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
1      b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
2      b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
3      b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
4      b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
                             ...                        
113    b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
114    b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
115    b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
116    b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
117    b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...
Name: AUDIO_FEATURES, Length: 118, dtype: object

In [24]:
tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
tDelete.start()
tDelete.join()

In [25]:
def extract_guide_per_row(row):
    print(row['ID'])
    image_guide_path = row['image_path']
    absPathVideo = row['VIDEO_PATH']
    #print(image_guide_path)
    image = Image.open(image_guide_path).convert('RGB')


    image = image.resize((begin_with_image_size,begin_with_image_size))
    im = image




    absPathGuide = absPathVideo.replace(datasetPathVideo,datasetPathGuides)
    #print(absPathGuide)
    absPathGuide = os.path.splitext(absPathGuide)[0]
    absPathGuide = absPathGuide + "_guide_" + str(row['ID']) + "_" + ".png"

    pathlib.Path(os.path.dirname(absPathGuide)).mkdir(parents=True, exist_ok=True) 
    image.save(absPathGuide)

    image.close()
    im.close()
    return absPathGuide

dataGotten['image_guide'] = dataGotten.apply(extract_guide_per_row,args=(),axis=1)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


In [26]:
dataGotten['image_guide']

0      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
1      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
2      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
3      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
4      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
                             ...                        
113    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
114    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
115    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
116    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
117    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
Name: image_guide, Length: 118, dtype: object

In [27]:
def save_dataframe_feature(row):
    #print(row)
    print(row['ID'])
    absPathVideo = row['VIDEO_PATH']
    absPathFeatures = absPathVideo.replace(datasetPathVideo,datasetPathFeatures)
    absPathFeatures = os.path.splitext(absPathFeatures)[0]
    absPathFeatures = absPathFeatures + "_features_"+ ".pickle"
    pathlib.Path(os.path.dirname(absPathFeatures)).mkdir(parents=True, exist_ok=True)
    with open(absPathFeatures, 'wb') as handle:
        pickle.dump(row, handle)
    return absPathFeatures

dataGotten['features_path'] = dataGotten.apply(save_dataframe_feature,args=(),axis=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


In [28]:
dataGotten['features_path']

0      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
1      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
2      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
3      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
4      /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
                             ...                        
113    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
114    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
115    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
116    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
117    /media/gamal/Passport/Datasets/VoxCeleb2Test/V...
Name: features_path, Length: 118, dtype: object

In [29]:
dataGotten.head(1)

,ID,FACE_ID,VIDEO_PATH,caption_a,caption_e,caption_g,SPEAKER_EMB,AUDIO_EMB,AUDIO_FEATURES,caption_l,image_path,VGG,image_guide,low_res_image,path_var_len_audio,features_path
0,0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,26,white,Man,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x83\x00\x00\x00\x00\x00\x00\x00...,English,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,"[0.02598828449845314, 0.002286622067913413, 0....",/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/home/gamal/vsc/DiffusionSpeech2Face/imagen_te...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...


In [30]:
dataGotten.isnull().sum().sum()

0

In [31]:
try:
    db.drop_table("video_stage1")
except:
    pass

def add_to_vector_db(row):
    print(row['ID'])
    absPathVideo = row['VIDEO_PATH']
    p_small = pathlib.Path(os.path.dirname(absPathVideo))
    p_big = p_small.parent.absolute()
    
    df_table = pd.DataFrame()
    df_table = df_table.append({'id': int(row['ID']), 'video_path': row['VIDEO_PATH'], 
                    'face_path': row['image_path'],'user':p_big.name,
        'blurred_face_path': row['image_guide'], 'features_path': row['features_path'], 
        'age': row['caption_a'],'gender':row['caption_g'],'ethnicity':row['caption_e'],
        'vector' : row['VGG'],'stage': 1}, ignore_index=True)
    #print(df_table)
    try:
        tbl = db.open_table("video_stage1")
        tbl.add(df_table)
        
    except:
        db.create_table("video_stage1", df_table)
        tbl = db.open_table("video_stage1")

    return df_table


dataGotten.apply(add_to_vector_db,args=(),axis=1)

try:
    shutil.rmtree(output_folder)
except:
    pass
tbl = db.open_table("video_stage1")
tbl.to_pandas()

0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117


,id,video_path,face_path,user,blurred_face_path,features_path,age,gender,ethnicity,vector,stage
0,0,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id00017,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,26,Man,white,"[0.025988284, 0.002286622, 0.0059057563, 0.008...",1
1,1,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id00061,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,31,Man,white,"[0.014909588, 0.01032118, 0.023922412, 0.00729...",1
2,2,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id00081,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,28,Man,latino hispanic,"[0.017716583, 0.031231219, 0.0167601, 0.001974...",1
3,3,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id00154,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,29,Man,middle eastern,"[0.015032666, 0.0020783008, 0.0017736976, -0.0...",1
4,4,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id00419,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,38,Woman,white,"[0.014704544, -0.0042701024, 0.00066350354, 0....",1
...,...,...,...,...,...,...,...,...,...,...,...
113,113,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id08552,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,26,Man,black,"[0.011644645, 0.006384785, 0.0013400487, 0.009...",1
114,114,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id08696,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,27,Man,black,"[0.01209747, 0.011594199, 0.018551253, 0.01129...",1
115,115,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id08701,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,44,Man,white,"[0.020973165, -0.011326933, -0.0015938727, 0.0...",1
116,116,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id08911,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,26,Man,white,"[0.027417418, -0.0014938384, 0.011837885, 0.00...",1


In [32]:
tbl = db.open_table("video_stage1")
tbl.search(np.random.rand(2622)).limit(2).to_df()

,id,video_path,face_path,user,blurred_face_path,features_path,age,gender,ethnicity,vector,stage,_distance
0,94,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id06913,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,25,Man,white,"[0.0050969413, 0.0027439885, 0.009167464, 0.00...",1,874.600281
1,47,/media/gamal/Passport/Datasets/Voxceleb2TestVi...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,id03347,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,/media/gamal/Passport/Datasets/VoxCeleb2Test/V...,31,Man,latino hispanic,"[0.0038344902, 0.0025192043, -0.0015666103, 0....",1,874.724731


: 